In [136]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [137]:
data = pd.read_csv("Results.csv", sep=',',encoding='utf-8')

fields = ['Pos','Bib No','Name','University','Time',
          'Cat Pos', 'Gender','Gen Pos','Run 1','G/Pos',
          'T1','Bike','G/Pos.1','T2','Run 2','G/Pos.2']

oxnames = []
oxidxs = []
camidxs = []
for idx in range(len(data)):
    if data["University"][idx] == 'OXFORD UNIVERSITY':
        oxnames.append(data['Name'][idx])
        oxidxs.append(idx)
    elif data["University"][idx] == 'UNIVERSITY OF CAMBRIDGE':
        camidxs.append(idx)

def secondCounter(timestr):
    timestr = str(timestr)
    if len(timestr) == 4:
        s = int(timestr[0:2])
    elif len(timestr) == 6:
        s = int(timestr[0])*60+int(timestr[2:4])
    elif len(timestr) == 7:
        s = int(timestr[0:2])*60+int(timestr[3:5])
    elif len(timestr) == 9:
        s = int(timestr[0])*3600 + int(timestr[2:4])*60+int(timestr[5:7])
    return s
    
    
def colToSeconds(field):
    dic = {}
    for i in range(len(data)):
        dic[i] = {field: secondCounter(data[field][i])}
    
    df = pd.DataFrame.from_dict(dic, "index")
    
    return df

for field in ["Run 1", "T1", "Bike", "T2", "Run 2", "Time"]:
    data[field] = colToSeconds(field)



In [165]:
def getRightData(name):
    idxs = []

    myidx = list(np.where(data["Name"] == name)[0])[0]
    for i in range(min(100,len(data)-myidx)):
        idxs.append(myidx+i-min(myidx//2,50))
        
    #idxs += oxidxs
    #idxs += camidxs
        
    results = data[fields].iloc[idxs]
    for field in ["Run 1",'T1', "Bike",'T2', "Run 2"]:
        avg = results[field].mean()
        results[field] = avg - results[field]
    
    
    results["Total after T1"] = results["Run 1"] + results["T1"]
    results["Total after Bike"] = results["Total after T1"] + results["Bike"]
    results["Total after T2"] = results["Total after Bike"] + results["T2"]
    results["Total after Run 2"] = results["Total after T2"] + results["Run 2"]

    myfigures(name, idxs, results)

def myfigures(name,idxs, results):
    
    x = ["Start","End of Run 1",'End of T1', "End of Bike",'End of T2', "End of Run 2"]
    
    

    plt.figure(4, figsize=(15,10))
    plt.title("Race Time")
    plt.ylabel("Time ahead relative to average")
    for idx in idxs:
        y = [0,results['Run 1'][idx], results['Total after T1'][idx], 
             results["Total after Bike"][idx], results['Total after T2'][idx], 
             results["Total after Run 2"][idx]]
        if results['Name'][idx] == name:
            plt.plot(x,y,'red',linewidth=1)
        elif results["University"][idx] == "OXFORD UNIVERSITY":
            plt.plot(x,y,'navy',linewidth=1)
        elif results["University"][idx] == "UNIVERSITY OF CAMBRIDGE":
            plt.plot(x,y,'#A3C1AD',linewidth=1)
        else:
            plt.plot(x,y,'gray', alpha=0.1,linewidth=1)
    plt.show()
    
    plt.figure(3, figsize=(15,10))
    for idx in idxs:
        y = [0,results['Run 1'][idx], results['T1'][idx]*10, 
             results["Bike"][idx], results['T2'][idx]*10, results["Run 2"][idx]]
        if results['Name'][idx] == name:
            plt.plot(x,y,'red',linewidth=1)
        elif results["University"][idx] == "OXFORD UNIVERSITY":
            plt.plot(x,y,'navy',linewidth=1)
        elif results["University"][idx] == "UNIVERSITY OF CAMBRIDGE":
            plt.plot(x,y,'#A3C1AD',linewidth=1)
        else:
            plt.plot(x,y,'gray', alpha=0.1,linewidth=1)
    plt.ylabel("Arbitraty measure of performance")
    plt.title("Individual discipline times")
    plt.show()

   





In [166]:
widgets.interact(getRightData, name=oxnames);

'''
How to interpret the graphs:

1. A flat line = average performance (of the 100 athletes around you). Positive gradient = better, negative = worse

2. Height of point = performance for that particular part of the race: e.g. select Mat Veal - a great bike, a bad T2
'''

interactive(children=(Dropdown(description='name', options=('Joe BOWNESS', 'Luke GRENFELL-SHAW', 'Andres NOE',…

'\nHow to interpret the graphs:\n\n1. A flat line = average performance (of the 100 athletes around you). Positive gradient = better, negative = worse\n\n2. Height of point = performance for that particular part of the race: e.g. select Mat Veal - a great bike, a bad T2\n'